# INGESTÃO DE DADOS SAÚDE SUPLEMENTAR ANS

## 1. IMPORT BIBLIOTECA

In [ ]:
import pandas as pd
from ftplib import FTP
from datetime import datetime
import os
from io import BytesIO
import zipfile
import unicodedata
import re
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
import pyodbc

## 2. CONFIGURAR ESTADOS E PATH BASE

In [ ]:
estados = ['PR', 'SC', 'RS']
path_base = '/FTP/PDA/informacoes_consolidadas_de_beneficiarios-024/'


### 3. CONFIGURAR CONEXÃO COM BANCO DE DADOS

In [ ]:
SERVER = 'ADIR'
DATABASE = 'BUSINESS'

In [12]:
conn_str = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={SERVER};'
    f'DATABASE={DATABASE};'
    'Trusted_Connection=yes;'           
    'Encrypt=yes;'                      
    'TrustServerCertificate=yes;'       
)


In [13]:
try:
    with pyodbc.connect(conn_str) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT @@version;")
        print("Conexão bem-sucedida!\nVersão do SQL Server:", cursor.fetchone()[0])
except Exception as e:
    print("Erro na conexão:", str(e))

Conexão bem-sucedida!
Versão do SQL Server: Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) 
	Oct  8 2022 05:58:25 
	Copyright (C) 2022 Microsoft Corporation
	Developer Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 26100: ) (Hypervisor)



In [27]:
ftp = FTP('dadosabertos.ans.gov.br')
ftp.login()

'230 Login successful.'

### 4. DEF PARA GERAR LISTA DOS ULTIMOS 12 MESES

In [ ]:

def gerar_lista_meses():
    hoje = datetime.now()
    meses =[]

    for i in range (12):
        ano = hoje.year
        mes= hoje.month - i

        if mes <= 0:
            mes += 12
            ano -= 1
        
        meses.append(f"{ano}{mes:02d}")
    
    return meses

In [15]:
meses = gerar_lista_meses()

In [16]:
meses

['202503',
 '202502',
 '202501',
 '202412',
 '202411',
 '202410',
 '202409',
 '202408',
 '202407',
 '202406',
 '202405',
 '202404']

## 5. LISTA PARA ARMAZENAR

In [ ]:
dfs = []

## 6. LOOP PARA BAIXAR ARQUIVOS

In [19]:

for periodo in gerar_lista_meses():
    try:
        ftp.cwd(path_base + periodo + '/')
        
        for estado in estados:
            nome_arquivo = f'pda-024-icb-{estado}-{periodo[:4]}_{periodo[4:]}.zip'
            
            try:
                buffer = BytesIO()
                ftp.retrbinary(f'RETR {nome_arquivo}', buffer.write)
                buffer.seek(0)
                
                with zipfile.ZipFile(buffer) as zip_ref:
                    with zip_ref.open(zip_ref.namelist()[0]) as arquivo_csv:
                        df = pd.read_csv(
                            arquivo_csv, 
                            sep=';', 
                            encoding='utf-8',
                            dtype={'CNPJ': 'str'},
                            low_memory=False
                        )
                        dfs.append(df)
                
                print(f" {periodo}/{estado} carregado")
                
            except Exception as e:
                print(f" Erro em {nome_arquivo}: {str(e)[:50]}...")
                
    except Exception as e:
        print(f" Diretório {periodo} não encontrado")

if dfs:
    df_final = pd.concat(dfs, ignore_index=True)
    
    print("\nResumo dos dados:")
    print(f"Total de registros: {len(df_final):,}")
    print(f"Estados encontrados: {df_final['SG_UF'].unique()}")
    print("\nPrimeiras linhas:")
    print(df_final.head())
else:
    print("Nenhum dado válido encontrado")

ftp.quit()

 Diretório 202503 não encontrado
 Diretório 202502 não encontrado
 202501/PR carregado
 202501/SC carregado
 202501/RS carregado
 202412/PR carregado
 202412/SC carregado
 202412/RS carregado
 202411/PR carregado
 202411/SC carregado
 202411/RS carregado
 202410/PR carregado
 202410/SC carregado
 202410/RS carregado
 202409/PR carregado
 202409/SC carregado
 202409/RS carregado
 202408/PR carregado
 202408/SC carregado
 202408/RS carregado
 202407/PR carregado
 202407/SC carregado
 202407/RS carregado
 202406/PR carregado
 202406/SC carregado
 202406/RS carregado
 202405/PR carregado
 202405/SC carregado
 202405/RS carregado
 202404/PR carregado
 202404/SC carregado
 202404/RS carregado

Resumo dos dados:
Total de registros: 25,962,177
Estados encontrados: ['PR' 'SC' 'RS']

Primeiras linhas:
  ID_CMPT_MOVEL  CD_OPERADORA  \
0       2025-01        371254   
1       2025-01        336831   
2       2025-01        416401   
3       2025-01        371254   
4       2025-01        304484   

'221 Goodbye.'

## 7. TRATAMENTO DE DADOS

In [20]:

df_final.isnull().sum()

ID_CMPT_MOVEL                 0
CD_OPERADORA                  0
NM_RAZAO_SOCIAL               0
NR_CNPJ                       0
MODALIDADE_OPERADORA          0
SG_UF                         0
CD_MUNICIPIO                  0
NM_MUNICIPIO                  0
TP_SEXO                       0
DE_FAIXA_ETARIA               0
DE_FAIXA_ETARIA_REAJ          0
CD_PLANO                     40
TP_VIGENCIA_PLANO             0
DE_CONTRATACAO_PLANO          0
DE_SEGMENTACAO_PLANO          0
DE_ABRG_GEOGRAFICA_PLANO      0
COBERTURA_ASSIST_PLAN         0
TIPO_VINCULO                  0
QT_BENEFICIARIO_ATIVO         0
QT_BENEFICIARIO_ADERIDO       0
QT_BENEFICIARIO_CANCELADO     0
DT_CARGA                      0
dtype: int64

In [21]:
df_final=df_final.dropna(subset=['CD_PLANO'])

In [28]:
df_final.head()

,ID_CMPT_MOVEL,CD_OPERADORA,NM_RAZAO_SOCIAL,NR_CNPJ,MODALIDADE_OPERADORA,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,TP_SEXO,DE_FAIXA_ETARIA,...,TP_VIGENCIA_PLANO,DE_CONTRATACAO_PLANO,DE_SEGMENTACAO_PLANO,DE_ABRG_GEOGRAFICA_PLANO,COBERTURA_ASSIST_PLAN,TIPO_VINCULO,QT_BENEFICIARIO_ATIVO,QT_BENEFICIARIO_ADERIDO,QT_BENEFICIARIO_CANCELADO,DT_CARGA
0,2025-01,371254,UNIMED REGIONAL MARINGÁ COOP.DE TRABALHO MÉDICO,76767219000182,COOPERATIVA MÉDICA,PR,411420,Mandaguari,M,15 a 17 anos,...,P,Coletivo Empresarial,Ambulatorial + Hospitalar com obstetrícia,Nacional,Médico-hospitalar,Dependente,1,0,0,2025-02-28
1,2025-01,336831,IRMANDADE SANTA CASA DE MISERICÓRDIA DE MARINGÁ,79115762000193,FILANTROPIA,PR,411520,Maringá,F,65 a 69 anos,...,P,Individual ou Familiar,Ambulatorial + Hospitalar com obstetrícia,Grupo de municípios,Médico-hospitalar,Dependente,15,0,0,2025-02-28
2,2025-01,416401,CAIXA DE ASSISTÊNCIA DOS EMPREGADOS DA EMPRESA...,8097092000181,AUTOGESTÃO,PR,411370,Londrina,M,65 a 69 anos,...,P,Coletivo Empresarial,Ambulatorial + Hospitalar com obstetrícia,Nacional,Médico-hospitalar,Titular,22,0,0,2025-02-28
3,2025-01,371254,UNIMED REGIONAL MARINGÁ COOP.DE TRABALHO MÉDICO,76767219000182,COOPERATIVA MÉDICA,PR,411750,Paiçandu,F,45 a 49 anos,...,P,Coletivo Empresarial,Ambulatorial + Hospitalar com obstetrícia,Grupo de municípios,Médico-hospitalar,Titular,12,0,1,2025-02-28
4,2025-01,304484,DENTAL UNI - COOPERATIVA ODONTOLÓGICA,78738101000151,COOPERATIVA ODONTOLÓGICA,PR,410580,Colombo,M,50 a 54 anos,...,P,Individual ou Familiar,Odontológico,Estadual,Odontológico,Titular,10,0,0,2025-02-28


In [23]:
df_final['NM_RAZAO_SOCIAL'].value_counts()

NM_RAZAO_SOCIAL
ODONTOPREV S/A                                            2030917
UNIMED PORTO ALEGRE - COOPERATIVA MÉDICA LTDA.            1035117
DENTAL UNI - COOPERATIVA ODONTOLÓGICA                     1033311
UNIMED NACIONAL - COOPERATIVA CENTRAL                     1025469
AMIL ASSISTÊNCIA MÉDICA INTERNACIONAL S.A.                 940946
                                                           ...   
SANTA RITA SISTEMA DE SAUDE LTDA                                2
CUIDAR.ME SAÚDE SA                                              2
SUL DO PARÁ LTDA                                                2
VIDA TOP MAIS SAÚDE OPERADORA DE PLANOS DE SAÚDE LTDA.          1
UNIMED MORRINHOS COOPERATIVA DE TRABALHO MEDICO                 1
Name: count, Length: 684, dtype: int64

In [24]:
df_final.dtypes

ID_CMPT_MOVEL                object
CD_OPERADORA                  int64
NM_RAZAO_SOCIAL              object
NR_CNPJ                       int64
MODALIDADE_OPERADORA         object
SG_UF                        object
CD_MUNICIPIO                  int64
NM_MUNICIPIO                 object
TP_SEXO                      object
DE_FAIXA_ETARIA              object
DE_FAIXA_ETARIA_REAJ         object
CD_PLANO                     object
TP_VIGENCIA_PLANO            object
DE_CONTRATACAO_PLANO         object
DE_SEGMENTACAO_PLANO         object
DE_ABRG_GEOGRAFICA_PLANO     object
COBERTURA_ASSIST_PLAN        object
TIPO_VINCULO                 object
QT_BENEFICIARIO_ATIVO         int64
QT_BENEFICIARIO_ADERIDO       int64
QT_BENEFICIARIO_CANCELADO     int64
DT_CARGA                     object
dtype: object

In [29]:
df_final.to_csv('dados.csv', sep=';', index=False, encoding='utf-8')

In [32]:
chunksize = 100000
for chunk in pd.read_csv('dados.csv', sep=';', chunksize=chunksize, encoding='utf-8'):
    dados = [tuple(row) for row in chunk.values]
    
    query = """
    INSERT INTO dbo.Staging_Beneficiarios (
        D_CMPT_MOVEL, 
        CD_OPERADORA, 
        NM_RAZAO_SOCIAL, 
        NR_CNPJ, 
        MODALIDADE_OPERADORA, 
        SG_UF, 
        CD_MUNICIPIO, 
        NM_MUNICIPIO, 
        TP_SEXO, 
        DE_FAIXA_ETARIA, 
        DE_FAIXA_ETARIA_REAJ, 
        CD_PLANO, 
        TP_VIGENCIA_PLANO, 
        DE_CONTRATACAO_PLANO, 
        DE_SEGMENTACAO_PLANO, 
        DE_ABRG_GEOGRAFICA_PLANO, 
        COBERTURA_ASSIST_PLAN, 
        TIPO_VINCULO, 
        QT_BENEFICIARIO_ATIVO, 
        QT_BENEFICIARIO_ADERIDO, 
        QT_BENEFICIARIO_CANCELADO, 
        DT_CARGA
    ) 
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    

    cursor = conn.cursor()
    cursor.fast_executemany = True
    cursor.executemany(query, dados)
    conn.commit()